In [1]:
# data gather through tweepy API
import pandas as pd
import time
import json
import csv
from tqdm import tqdm
import tweepy
import requests
import os
import numpy as np
api = pd.read_csv(r'C:\Users\63014\Desktop\twitter key.txt')
consumer_key        = api.iloc[0,1]
consumer_secret     = api.iloc[1,1]
access_token        = api.iloc[3,1]
access_token_secret = api.iloc[4,1]
bearer_token        = api.iloc[2,1]

In [2]:
auth = tweepy.OAuthHandler(consumer_key,consumer_secret)
auth.set_access_token(access_token,access_token_secret)
api = tweepy.API(auth,wait_on_rate_limit=True)
client = tweepy.Client(bearer_token=bearer_token)

In [18]:
df = pd.read_csv('../../data/NAACL_SRW_2016.csv')
df.columns = ['user_id','label']
# 有重复的号码
df = df.drop_duplicates(subset='user_id',keep='first').reset_index(drop=True)
df.head()
print(df.shape)
print(np.unique(df.label))

(16849, 2)
['none' 'racism' 'sexism']


In [8]:
def tqdm_enumerate(iter):
    i = 0
    for y in tqdm(iter):
        yield i, y
        i += 1

In [25]:
delete = []
text_list = []
hashtags_list = []
id_list = []
location_list = []
followers_count_list = []
friends_count_list = []
country_list = []
retweet_count_list = []
favorite_count_list = []
flag = True
for count, i in tqdm_enumerate(iter=df.user_id):
    try:
        status = api.get_status(id=str(i))._json
        count += 1
    except:
        delete.append(i)
        continue
    else:
        text = status['text']
        hashtags = status['entities']['hashtags']
        if hashtags.__len__() == 0:
            hashtext = np.NaN
        else:
            hashtext = hashtags[0]['text']
        id = status['user']['id']
        location = status['user']['location']
        followers_count = status['user']['followers_count']
        friends_count = status['user']['friends_count']
        try:
            country = status['place']['country']
        except:
            country = np.NaN
        retweet_count = status['retweet_count']
        favorite_count = status['favorite_count']
        text_list.append(text)
        hashtags_list.append(hashtext)
        id_list.append(id)
        location_list.append(location)
        followers_count_list.append(followers_count)
        friends_count_list.append(friends_count)
        country_list.append(country)
        retweet_count_list.append(retweet_count)
        favorite_count_list.append(favorite_count)
    if count % 25 == 0:
        time.sleep(1)
        print('we need to sleep to avoid reaching limit')
    elif count % 100 == 0:
        time.sleep(5)
        print('we need to sleep more to avoid reaching limit')
    elif count % 500 == 0:
        df_temp = pd.DataFrame.from_records({'user_id': id_list, 'text': text_list, 'hashtag': hashtags_list, 'location': location_list, 'follower_count': followers_count_list,
                                                'friends_count': friends_count_list, 'country': country_list, 'retweet_count': retweet_count_list, 'favorite_count': favorite_count_list})
        df_temp.to_csv('./hate_speech_test.csv', 'a')
        break


  1%|          | 86/16849 [00:39<2:34:26,  1.81it/s]

In [22]:
df_clean.loc[:,'id'] = id_list
df_clean.loc[:,'text'] = text_list
df_clean.loc[:,'hashtag'] = hashtags_list
df_clean.loc[:,'location'] = location_list
df_clean.loc[:,'follower_count'] = followers_count_list
df_clean.loc[:,'friends_count'] = friends_count_list
df_clean.loc[:,'country'] = country_list
df_clean.loc[:,'retweet_count'] = retweet_count_list
df_clean.loc[:,'favorite_count'] = favorite_count_list

0

In [15]:
with open('hate_speech.csv','a') as fd:
    for count,i in tqdm_enumerate(iter=df.user_id):
        try:
            status = api.get_status(id=str(i))._json
        except:
            continue
        else:
            text = status['text']
            hashtags = status['entities']['hashtags']
            if hashtags.__len__()==0:
                hashtext = np.NaN
            else:
                hashtext = hashtags[0]['text']
            id = status['user']['id']
            location = status['user']['location']
            followers_count = status['user']['followers_count']
            friends_count = status['user']['friends_count']
            try:
                country = status['place']['country']
            except:
                country = np.NaN
            retweet_count = status['retweet_count']
            favorite_count = status['favorite_count']
            df_temp = pd.DataFrame.from_records()


,user_id,label,text
0,562874396155850752,none,RT @randi_ebooks: Twitter just switched to mon...
1,470393182505545728,none,Because if Elliot Rodger had killed just ONE w...
4,567937595390300160,none,@Grummz @rxptor @Spacekatgal @TheQuinnspiracy ...
8,575948706039492609,none,My face every time I hear or see Kat....hope i...
10,537647038649741312,none,Empathy is the social skill that informs all o...
...,...,...,...
2521,572330892972068864,none,Beans on toast for dinner tonight. Would score...
2522,564099641604861952,sexism,Is it lost on most #QuestionsForMen was essent...
2523,566067783457591297,none,Step 1: post to facebook.\nStep 2: KiA links t...
2524,571499576152231936,sexism,RT @fruitondabottom: @MT8_9 Didn't you hear ab...


In [17]:
df_clean = df.query('user_id not in @delete').copy()
df_clean.loc[:,'id'] = id_list
df_clean.loc[:,'text'] = text_list
df_clean.loc[:,'hashtag'] = hashtags_list
df_clean.loc[:,'location'] = location_list
df_clean.loc[:,'follower_count'] = followers_count_list
df_clean.loc[:,'friends_count'] = friends_count_list
df_clean.loc[:,'country'] = country_list
df_clean.loc[:,'retweet_count'] = retweet_count_list
df_clean.loc[:,'favorite_count'] = favorite_count_list

In [40]:
df_clean.sample(n=5)

,user_id,label,text,hashtag,id,location,follower_count,friends_count,country,retweet_count,favorite_count
1643,572338814716272640,none,New #MKR format is 20Twenty... 20 teams in twe...,MKR,42542097,Shoalhaven,348,1169,NaN,0,0
337,572364747674292225,sexism,Girls claiming they're hot generally have an o...,mkr,1523942082,"Perth, Western Australia",1194,1736,NaN,0,1
2429,573180997631537152,none,@GentlerMouse http://t.co/qYgaRbGQDP,NaN,13857342,"Portland, OR",19172,797,NaN,0,0
1883,575957444112748546,none,This humiliates you? I wonder what she thinks ...,MKR,347048522,Sydney,2650,2497,NaN,0,1
461,561984177701421056,none,RT @g56yu: @PierceCotwa is now on twitter. If ...,NaN,2756873076,Earth,1796,1858,NaN,2,0


In [41]:
df_clean.to_csv('./hate_speech.csv')